[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/90_pixel_inspector.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/90_pixel_inspector.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

## Interactive pixel inspector 

The interactive pixel inspector can be used to explore the pixel values of an image. It supports Cloud Optimized GeoTIFF (COG), STAC, and other raster data formats, either stored locally or on the cloud. The COG and STAC functionalities are powered by the [TiTiler](https://developmentseed.org/titiler/), while the local file support is powered by [localtileserver](https://github.com/banesullivan/localtileserver).

In [ ]:
# %pip install "leafmap[raster]"

In [ ]:
import leafmap
import rasterio
import rioxarray
import xarray as xr

### COG

In [ ]:
m = leafmap.Map()
url = "https://github.com/opengeos/data/releases/download/raster/Libya-2023-07-01.tif"
m.add_cog_layer(url, name="Libya")
m.add("inspector")
m

### STAC

In [ ]:
m = leafmap.Map()
url = "https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_11055_6057_20070622/S5_11055_6057_20070622.json"
m.add_stac_layer(url, bands=["B3", "B2", "B1"], name="SPOT Image")
m.add("inspector")
m

### Planetary Computer

In [ ]:
m = leafmap.Map()
collection = "landsat-8-c2-l2"
item = "LC08_L2SP_047027_20201204_02_T1"
m.add_stac_layer(
    collection=collection,
    item=item,
    assets="SR_B7,SR_B5,SR_B4",
    name="Landsat Band-754",
)
m.add("inspector")
m

### Local raster

In [ ]:
url = "https://open.gishub.org/data/raster/landsat.tif"
satellite = leafmap.download_file(url, "landsat.tif", overwrite=True)

In [ ]:
m = leafmap.Map()
m.add_raster(satellite, indexes=[4, 1, 2], vmin=0, vmax=120, layer_name="Landsat 7")
m.add("inspector")
m

## In-memory raster

### NumPy array

In [ ]:
dataset = rasterio.open(satellite)
nir = dataset.read(4).astype(float)
red = dataset.read(1).astype(float)
ndvi = (nir - red) / (nir + red)
ndvi_image = leafmap.array_to_image(ndvi, source=satellite)

In [ ]:
m = leafmap.Map()
m.add_raster(satellite, indexes=[4, 1, 2], vmin=0, vmax=120, layer_name="Landsat 7")
m.add_raster(ndvi_image, colormap="Greens", layer_name="NDVI")
m.add("inspector")
m

### Xarray DataArray

In [ ]:
url = "https://open.gishub.org/data/raster/srtm90.tif"
dem = leafmap.download_file(url, "srtm90.tif")

In [ ]:
ds = rioxarray.open_rasterio(dem)
ds

In [ ]:
array = ds.sel(band=1)
masked_array = xr.where(array < 2000, 0, 1)

In [ ]:
m = leafmap.Map()
m.add_raster(dem, colormap="terrain", layer_name="DEM")
m.add_raster(masked_array, colormap="coolwarm", layer_name="Classified DEM")
m

## Split map

In [ ]:
m = leafmap.Map(center=[37.6, -119], zoom=9)
m.split_map(
    dem,
    masked_array,
    left_args={
        "layer_name": "DEM",
        "colormap": "terrain",
    },
    right_args={
        "layer_name": "Classified DEM",
        "colormap": "coolwarm",
    },
)
m

![](https://i.imgur.com/2AduU8G.gif)